In [7]:
import torch
import torch.nn as nn
from transformers import CLIPVisionModelWithProjection, ViTForImageClassification, AutoModelForCausalLM
from transformers import AutoModel, AutoTokenizer

import sys, os, json
from tqdm import tqdm

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

if project_root not in sys.path:
    sys.path.append(project_root)

from VQ_SEEDLM import models

In [13]:
def reconstruct_model(state_dict, model, weight_condition, batch_size=32768):
    with torch.no_grad():
        mean_MSE = 0
        count = 0
        mse_func = nn.MSELoss()
        device = next(model.parameters()).device
        recon_state_dict = {}
        
        for k, W in state_dict.items():
            if not weight_condition in k: continue
            print(f'### Reconstructing {k} ####')
            
            W_reshaped = W.reshape(-1, model.input_size) # ( -1, -1) --> (-1, size, size)
            W_recon = torch.zeros(W_reshaped.shape, dtype=W_reshaped.dtype, device=W_reshaped.device)
            
            for start_idx in tqdm(range(0, W_reshaped.shape[0], batch_size)):
                end_idx = min(start_idx + batch_size, W_reshaped.shape[0])  # 마지막 배치를 처리할 때 범위 조정
                batch = W_reshaped[start_idx:end_idx]  # batch_size 크기로 슬라이싱
                batch = batch.to(device)  # 배치를 GPU로 이동

                out = model(batch)
                x_hat = out['x_hat']
                W_recon[start_idx:end_idx] = x_hat

                mean_MSE += mse_func(out["x"], out["x_hat"]).item()
                count += 1

            W_recon = W_recon.reshape(W.shape).cpu()
            recon_state_dict[k] = W_recon
            
        mean_MSE /= count  

    return recon_state_dict, mean_MSE

In [10]:
model_path = '/home/jgryu/Weight_compression/VQ_SEEDLM/checkpoint/Meta-Llama-3-8B/mlp_16_row_dataset.pt/size16_ne512_P4_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.11122_total_iter_2000000.pth.tar'
ckpt = torch.load(model_path)

with open('/home/jgryu/Weight_compression/Wparam_dataset/dataset_per_row/meta-llama/Meta-Llama-3-8B/mlp_16_row_dataset_stats.json', 'r', encoding='utf-8') as file:
        dataset_stats = json.load(file)  # JSON 파일을 Python 객체로 변환

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

input_size = 16
dim_encoder = 64
P = 4
ne = 512
n_resblock = 4

model = models.VQ_SEEDLM(input_size = 16, 
                    dim_encoder = 64, 
                    P = 4, n_embeddings = 512, n_resblock = 4, 
                    beta = 0.25,
                    scale = torch.Tensor(dataset_stats['train']['mean_channel']).to(device), 
                    shift = torch.Tensor(dataset_stats['train']['mean_channel']).to(device)
                    )

model.load_state_dict(ckpt['state_dict'])
model.to(device)

ckpt_path = '/home/jgryu/Weight_compression/model_cache/models--meta-llama--Meta-Llama-3-8B/snapshots/8cde5ca8380496c9a6cc7ef3a8b46a0372a1d920'
net = AutoModelForCausalLM.from_pretrained(ckpt_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(ckpt_path, local_files_only=True)
state_dict = net.state_dict()

/tmp/ipykernel_2100137/3131610841.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path)
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00

In [ ]:
recon_state_dict, mean_MSE = reconstruct_model(
        state_dict, model, weight_condition = 'mlp')

print(mean_MSE)

for k, v in state_dict.items():
    if k not in recon_state_dict.keys():
        recon_state_dict[k] = v
        print(k, v.shape)
    else:
        assert recon_state_dict[k].shape == state_dict[k].shape

### Reconstructing model.layers.0.mlp.gate_proj.weight ####


  0%|          | 0/112 [00:00<?, ?it/s]

100%|██████████| 112/112 [00:06<00:00, 16.51it/s]


### Reconstructing model.layers.0.mlp.up_proj.weight ####


100%|██████████| 112/112 [00:06<00:00, 17.37it/s]


...

100%|██████████| 112/112 [00:07<00:00, 15.74it/s]


### Reconstructing model.layers.31.mlp.down_proj.weight ####


100%|██████████| 112/112 [00:06<00:00, 16.19it/s]

0.0006984801366911597


NameError: name 'reconstruncted_state_dict' is not defined

In [15]:
for k, v in state_dict.items():
    if k not in recon_state_dict.keys():
        recon_state_dict[k] = v
        print(k, v.shape)
    else:
        assert recon_state_dict[k].shape == state_dict[k].shape

model.embed_tokens.weight torch.Size([128256, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([1024, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.input_layernorm.weight torch.Size([4096])
model.layers.1.post_attention_layernorm.weight torch.Size([4096])
model.layers.2.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.2.self_attn.k_proj.weight torch.Size([1024, 4096])
model.layers.2.self_attn.v_proj.weight torch.Size([1024, 4096

In [22]:
net.load_state_dict(recon_state_dict)
save_directory = f"/home/jgryu/Weight_compression/model_cache_reconstructed/vq_seedlm/{os.path.join(*model_path.split('/')[-3:])}"
net.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/home/jgryu/Weight_compression/model_cache_reconstructed/vq_seedlm/mlp_16_row_dataset.pt/size16_ne512_P4_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.11122_total_iter_2000000.pth.tar/tokenizer_config.json',
 '/home/jgryu/Weight_compression/model_cache_reconstructed/vq_seedlm/mlp_16_row_dataset.pt/size16_ne512_P4_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.11122_total_iter_2000000.pth.tar/special_tokens_map.json',
 '/home/jgryu/Weight_compression/model_cache_reconstructed/vq_seedlm/mlp_16_row_dataset.pt/size16_ne512_P4_batch_size512_total_iter2000000_lr0.0001_seed100/best_mse_model_MSE_0.11122_total_iter_2000000.pth.tar/tokenizer.json')